In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog
from tkinter.messagebox import Message 
from datetime import datetime
import sys
date = datetime.today().strftime('%Y-%m-%d')
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

if int(date[:4]) > 2023:
    tk.messagebox.showinfo(title='Message', message='Program need to update, please contact Surfscan NPI Team',)
    sys.exit()
elif int(date[:4]) > 2022 and int(date[5:7]) > 11:
    tk.messagebox.showinfo(title='Message', message='Program is going to out of date, please contact Surfscan NPI Team')

In [ ]:
from selenium import webdriver
# import pandas as pd
# import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import win32api
import string
import os
import re
# import fnmatch
# import shutil
# import pathlib
from time import sleep
# import threading
# import multiprocessing
# import concurrent.futures
import urllib.request
import random
# import win32com.client
# from openpyxl import load_workbook
# import fitz
# from fitz import Document, Page, Rect
# import xlwings as xw
# from xlwings.constants import DeleteShiftDirection
from selenium.webdriver.support.ui import WebDriverWait as wait
from contextlib import suppress
import threading


In [ ]:
# function to filter dash roll
def pure_name(matched): # for part number find
    if matched.group(0) == '-':
        return '-'
    else:
        return ''

def pure_qty(matched): # for part number find
    if matched.group(0) == ',' or matched.group(0) == '.':
        return '.'
    else:
        return ''

In [ ]:
part_pattern = re.compile(r'\b\d+[A-Z]*\d+[-]*\d*[A-Z]*\d+[-]*[A-Z]*\d+[-]*[A-Z]*\d+\b')
#     re.compile(r'\b\d{7}[-_ ]\d{3}\b') #(r'[\W?]\d{7}[-_]\d{3}[-_]?[A-Z]?[A-Z]?[\W]')
dash_pattern = re.compile(r'[oO]\W+[_]?\d\d\d?[\W?]') #re.compile(r'o\W+[_]?\d{3}[\W?]')
rev_pattern = re.compile(r'[oO]\W*?[-_ ][A-Z][A-Z]\b')#(r'o\W+[_-]?[A-Z][A-Z][\W?]')
des_pattern = re.compile(r'[\(\[].{5,45}[\)\]]')
#     des_pattern = re.compile(r'\(.{5,45}\)') #re.compile(r'[\(\[].{5,45}[\)\]]')
qty_change = re.compile(r'[oO]\W+\d{0,5}[,.]?\d*',re.IGNORECASE)
qty_add = re.compile(r'\W+\d{1,5}[,.]?\d*',re.IGNORECASE)
sub_pur_roll = r'[\W+a-z_O]'

#     rev_pattern = re.compile(r'o\W*?[-_ ][A-Z][A-Z]\b')

In [ ]:
# extract and get data from a line then return [[pn,procedure,tfw,header,par_child],...]
def extract_def(line):
# for line in lines:
    cannot_find = []
    return_list = []
#     part_pattern = re.compile(r'\b[A-Z]*?\d*[A-Z]*?\d+[-]*?\d+[A-Z]*?\d+[-]*\d+[-]*[A-Z]*?\d+\b')
    
    if len(line) < 15:
        return []
    raw_parts = part_pattern.findall(line)
#     raw_des = des_pattern.findall(line)
    raw_dash = dash_pattern.findall(line)
    raw_rev = rev_pattern.findall(line)

        
#     qty_number = re.compile(r'qty\W+?\d+[f?F?\W?M?m?]+')
    pn = ''
    procedure = ''
    tfw = ''
    header = ''
    par_child = ''
#     print(line)
#     print(raw_parts,raw_dash,raw_rev)
    if line.strip()[0].isdigit():
#         print(line[0], 'isdigit',line)
        header = 'header'
    if 'below' in line.lower() and 'bom' in line.lower():
        par_child = 'par'
    elif header == 'header' and ('reflect' in line.lower() or 'capture' in line.lower()):
        par_child = 'child'
    else:
        par_child = ''
        
#     if (len(raw_parts) > 1) and ('replace' not in line.lower()): #\
    # combine line from 1 part 
    if (len(raw_parts) > 0): # and ('replace' not in line.lower()): #\
#         and ('reflect' in line.lower()):
#         par_child = 'child'
#         print(f'has {len(raw_parts)} parts, {raw_parts},{raw_dash},{raw_rev}')
#         if len(raw_parts) == len(raw_dash) + len(raw_rev): # check it can remove or not
        for i in range(len(raw_parts)):
            pn_i1 = ''
#             print(i,len(raw_parts),'raw pn',raw_parts[i])
            pn = re.sub('[\W+a-z]',pure_name,raw_parts[i]) #[:11]
            
            if i == len(raw_parts) - 1:
        #                     print('index',i,line.find(pn),'len line:',len(line))
#                     pn_i1 = str(len(line))
                end_find = len(line)
            else:
                pn_i1 = re.sub('[\W+a-z]',pure_name,raw_parts[i+1]) #[:-3] #[:11]
                end_find = line.find(pn_i1)
                
            
                
            
#             pn_i = re.sub('[\W+a-z]',pure_name,raw_parts[i]) #[:-3] #[:11]
            if i == 0:
                start_find = 1
            else:
                start_find = line.find(pn)  + len(pn) # + 3
#             print('start_find',start_find,'end_find',end_find)
            raw_dash = dash_pattern.findall(line[start_find:end_find])
            raw_rev = rev_pattern.findall(line[start_find:end_find])
#             print(i,'pure_name,raw_parts[i])',pn,'dash/rev',raw_dash,'/',raw_rev)
#             if len(raw_dash) > 0 and 'dash' in line[start_find:end_find].lower():
            if 'dash' in line[start_find:end_find].lower() and 'roll' in line[start_find:end_find].lower():
                procedure = 'dash'
                if len(raw_dash) > 0:
#                     print(f'{pn} is dash with raw_dash {raw_dash}')
                    dash = re.sub(sub_pur_roll,'',raw_dash[0])
#                     print('0001-1: ',pn.split('-')[-1].isnumeric(),int(dash) < int(pn.split('-')[-1]) +1)
                    if pn.split('-')[-1].isnumeric() and int(dash) < int(pn.split('-')[-1]) +1:
#                         print('new dash not geater than current',pn)
                        dash = int(pn.split('-')[-1]) + 1
                        tfw = pn[:-len(str(dash))] + str(dash)
    #                     tfw = pn[:8] + str(dash)
                    else:
#                         print('dash is not numeric or new dash geater than current',pn)
                        tfw = pn[:-len(str(int(dash)))] + str(int(dash))
                ### check if: 1234567-000 is dash rolled to 1234567-001 
                elif pn[:-3] == pn_i1[:-3]:
                    print(i,pn, ' in line format: ',line)
                    tfw = pn_i1
                    return_list.append([pn,procedure,tfw,header,par_child])
                    break
                
                else:
#                     print(f'{pn} is dash without raw_dash {raw_dash}')
                    dash = pn.split('-')[-1]
                    if dash.isnumeric():
                        dash = int(dash) + 1
                        tfw = pn[:-len(str(dash))] + str(dash)
                    else:
                        tfw = 'NOT_DASH'
#             elif len(raw_rev) > 0 and 'rev' in line[start_find:end_find].lower():
            elif 'rev' in line[start_find:end_find].lower() and 'roll' in line[start_find:end_find].lower():
                procedure = 'rev'
                if len(raw_rev) < 1:
                    tfw = re.sub(sub_pur_roll,'','not found rev')
                else:
                    tfw = re.sub(sub_pur_roll,'',raw_rev[0])

            elif 'release' in line.lower() and not ('pre-release' in line.lower()\
                or 'pre release' in line.lower() or 'prerelease' in line.lower()\
                        or 'pre_release' in line.lower() or 'pre/release' in line.lower()):
#             print(line,'release')
                procedure = 'release'
                tfw = ''
            elif 'inactive' in line.lower():
    #             print(line,'inactive')
                procedure = 'inactive'
                tfw = ''
            elif 'replace' in line.lower() and len(raw_parts) == 2:
                procedure = 'replace'
                tfw =  re.sub('[\W+a-z]',pure_name,raw_parts[1])
                return_list.append([pn,procedure,tfw,header,par_child])
                break
            elif 'remove' in line.lower():
                procedure = 'remove'
            elif 'add' in line.lower():
                procedure = 'add'
                qty_letter = re.search(r'qty',line,re.IGNORECASE)
                if qty_letter is not None:
                    start_f = qty_letter.end()
                    qty_num = qty_add.findall(line[start_f:])
                    if len(qty_num) == 1:
                        tfw = re.sub(sub_pur_roll,pure_qty,qty_num[0])
                    else:
                        print('cannot found qty to add', line)
                        tfw = ''
                else:
                    tfw = ''
            elif ('change' in line.lower()) and ('qty' in line.lower() or 'quantity' in line.lower()):
                procedure = 'change'
                qty_letter = re.search(r'qty',line,re.IGNORECASE)
                if qty_letter is not None:
                    start_f = qty_letter.end()
                    qty_num = qty_change.findall(line[start_f:])
                if len(qty_num) == 1:
                    tfw = re.sub(sub_pur_roll,pure_qty,qty_num[0])
                else:
                    print('cannot found qty to change',line)
                    tfw = ''
            elif len(raw_parts) == 1 and not (\
                ('change' in line.lower())\
                or ('release' in line.lower())\
                or ('inactive' in line.lower())\
                or ('remove' in line.lower())\
                or ('add' in line.lower())\
                or ('roll' in line.lower())\
                                             ):
    #             print(raw_parts)
                pn = re.sub('[\W+a-z]',pure_name,raw_parts[0])[:11]
            
            
            
            return_list.append([pn,procedure,tfw,header,par_child])

#         else:
#             print(line,'not match part and roll, need to improve code')
#             return
    elif len(raw_parts) < 1:
        if 'release' in line.lower() and not ('pre-release' in line.lower()\
            or 'pre release' in line.lower() or 'prerelease' in line.lower()\
                        or 'pre_release' in line.lower() or 'pre/release' in line.lower()) :
#             print('release line', line)
            procedure = 'release'
            return_list.append([pn,procedure,tfw,header,par_child])
        elif 'inactive' in line.lower():
#             print('inactive line', line)
            procedure = 'inactive'
            return_list.append([pn,procedure,tfw,header,par_child])
        else:
            print('can not extract for line:',line)
#     else:
#         pn = re.sub('[\W+a-z]',pure_name,raw_parts[0])[:11]
#         if 'dash' in line.lower() and len(raw_dash) > 0:
#             procedure = 'dash'
#             dash = re.sub('[\W+a-z]','',raw_dash[0])
# #             if int(dash) != int(pn[8:]) + 1:
# # #                 print('new dash roll must be current + 1, check dash roll for:\n',line)
# #                 tfw = pn[:8] + str(dash)
# #             else:
#             tfw = pn[:8] + str(dash)
#         elif 'rev' in line.lower() and len(raw_rev) > 0:
#             procedure = 'rev'
#             tfw = re.sub('[\W+a-z]','',raw_rev[0])

#         elif 'release' in line.lower() and not ('pre-release' in line.lower()\
#             or 'pre release' in line.lower() or 'prerelease' in line.lower()\
#                         or 'pre_release' in line.lower()) :
# #             print(line,'release')
#             procedure = 'release'
#             tfw = ''
#         elif 'inactive' in line.lower():
# #             print(line,'inactive')
#             procedure = 'inactive'
#             tfw = ''
#         elif 'replace' in line.lower() and len(raw_parts) == 2:
#             procedure = 'replace'
#             tfw =  re.sub('[\W+a-z]',pure_name,raw_parts[1])[:11]
#         elif 'remove' in line.lower():
#             procedure = 'remove'
#         elif 'add' in line.lower():
#             procedure = 'add'
#             qty_letter = re.search(r'qty',line,re.IGNORECASE)
#             if qty_letter is not None:
#                 start_f = qty_letter.end()
#                 qty_num = qty_add.findall(line[start_f:])
#                 if len(qty_num) == 1:
#                     tfw = re.sub('[\W+a-z]',pure_qty,qty_num[0])
#                 else:
#                     print('cannot found qty to add', line)
#                     tfw = ''
#             else:
#                 tfw = ''
#         elif ('change' in line.lower()) and ('qty' in line.lower() or 'quantity' in line.lower()):
#             procedure = 'change'
#             qty_letter = re.search(r'qty',line,re.IGNORECASE)
#             if qty_letter is not None:
#                 start_f = qty_letter.end()
#                 qty_num = qty_change.findall(line[start_f:])
#             if len(qty_num) == 1:
#                 tfw = re.sub('[\W+a-z]',pure_qty,qty_num[0])
#             else:
#                 print('cannot found qty to change',line)
#                 tfw = ''
#         elif len(raw_parts) == 1 and not (\
#                 ('change' in line.lower())\
#                 or ('release' in line.lower())\
#                 or ('inactive' in line.lower())\
#                 or ('remove' in line.lower())\
#                 or ('add' in line.lower())\
#                 or ('roll' in line.lower())\
#                                          ):
# #             print(raw_parts)
#             pn = re.sub('[\W+a-z]',pure_name,raw_parts[0])[:11]
# #             print('cannot found procedure',line)
#         return_list.append([pn,procedure,tfw,header,par_child])  

    return return_list
#     break
# print(return_list)
# with open(script_file) as f:
#     lines = f.readlines()
# # # part_pattern = re.compile(r'\b[A-Z]*\d+[A-Z]*\d+[-]*\d+[A-Z]*\d+[-]*\d+[-]*\d+\b')
# # # dash_pattern = re.compile(r'o\W+[_]?\d\d\d?[\W?]')
# # # rev_pattern = re.compile(r'o\W*?[-_ ][A-Z][A-Z]\b')
# i = 1
# for line in lines:
    
# #     raw_parts = part_pattern.findall(line)
# #     raw_dash = dash_pattern.findall(line)
# #     raw_rev = rev_pattern.findall(line)
#     print(i,extract_def(line))
#     i += 1

# with open(script_file) as f:
#     lines = f.readlines()
# for line in lines:
# #     print('line',line)
#     print('extract return',extract_def(line))

In [ ]:
def find_eng_name(df_input,msg_message):
    if len(df_input) > 1:
        name_found = False
        for name in df_input:
            if name.lower() in msg_message.lower():
                eng_name = name
                name_found = True
                break
        if name_found == False:
            eng_name = ''
    else:
        eng_name = df_input[0]
    return eng_name

# Select file (request file or script file)

In [ ]:
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
file_full = filedialog.askopenfilename(title= "Please select Script file")
if file_full =='':
    print('file none, exit program')
    sys.exit()
file_type = os.path.splitext(file_full)[1]
file_path = os.path.dirname(file_full)
file_name = os.path.basename(file_full)
down_folder = os.path.expanduser('~') + '\Downloads'
eco_num = simpledialog.askstring(title="ECO Number",
                              prompt="Please input ECO number:")
if eco_num is not None:
    if eco_num.isnumeric() and len(eco_num) == 5:
        eco_num = 'D' + str(eco_num)

    elif len(eco_num) < 5:
        eco_num = None
    print('ECO:',eco_num)
else:
    print('Not input the ECO number')
    sys.exit()
script_file = file_full

In [ ]:
# def extract_script_to_list(line, pn_list):
#     part_pur = ''
#     rev_pur = ''
#     des_raw = ''
    
# #     des_pattern = re.compile(r'\(.{5,45}\)')pn = re.sub('[\W+a-z]',pure_name,raw_parts[i])
    
#     if 'roll' in line.lower():
#         part_raw = part_script_pattern.findall(line)
#         des_raw = des_pattern.findall(line)
#         if 'dash' in line.lower():
#             dash_rolled = int(part_raw[0][8:]) + 1
#             part_pur = part_raw[0][:8] + str('{:03d}'.format(dash_rolled))
#             rev_pur = 'AA'
#         elif 'rev' in line.lower():
#             part_pur = part_raw[0][:11]
#             if len(part_raw) > 12:
#                 rev_roll = rev_pattern.findall(line)[0]
#                 rev_pur = re.sub(sub_pur_roll,'',rev_roll)
#             else:
#                 rev_pur = ''
#         else:
#             rev_pur = ''
# #                     print('part:',part_pur,rev_pur)
#         if len(part_raw) != 1: # or len(des_raw) != 1:
#             print('can not found part or description')
#         else:
# #                     if len(pn_list) > 1:
#             if not any(part_raw[0][:7] in word for word in pn_list):
# #                 print(part_pur,rev_pur,'not found in list',pn_list)
#                 pn_list.append(part_raw[0][:11])
#     return pn_list, part_pur, rev_pur, des_raw

In [ ]:
def get_eco_win(eco_num):

#     main_page = driver.current_window_handle
    sleep(random.randint(2, 3))
    driver.switch_to.window(main_page)
#     driver.maximize_window()
    try:
        btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                        '//*[@id="tvc-bin-icon"]/i')))
        sleep(random.randint(2, 3))
    except TimeoutException:
        driver.refresh()
        sleep(random.randint(5, 7))
        try:
            print('try 2nd time to get Enovia site after refresh')
            btn_plus = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                            '//*[@id="tvc-bin-icon"]/i')))
            sleep(random.randint(2, 3))
        except:
            tk.messagebox.showinfo(title='Error 101', \
            message='Can not load Enovia site, check your connection and try again')
            sys.exit()
        sleep(random.randint(2, 3))
    #     if len(driver.window_handles) > 1:
    #             print(main_page)
    #         for windown in driver.window_handles:
    #             if windown != main_page:
    #                 driver.switch_to.window(windown)
    #                 driver.close()
    driver.switch_to.window(main_page)
    search_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
                                                        '//*[@id="quickSearch"]')))
    search_field.send_keys(eco_num)
    #     sleep(random.randint(0, 1))
    search_field.send_keys(Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    #     driver.switch_to.frame('tableContentFrame')
    driver.switch_to.frame('tableBodyRight')
    ecos = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    if len(ecos) == 1:
    #     print('one ECO found')
        eco_select = driver.find_element(By.XPATH, '//*[@id="tbr"]/tbody/tr/td/a')
    else:
        print('no or more than one ECO found')
        eco_select_number = simpledialog.askstring(title="Script to Enovia",
                                      prompt="Please select ECO, input 1, 2 or 3:")
        eco_select = driver.find_element(By.XPATH,\
                    '//*[@id="tbr"]/tbody/tr[' + str(eco_select_number) + ']/td/a')
    eco_select.click()
    sleep(5)
    driver.switch_to.window(driver.window_handles[-1])
    eco_win = driver.current_window_handle
#     print(eco_win)
    return eco_win

In [ ]:
# affected windown select
def get_affected_win(eco_win):
    driver.switch_to.window(eco_win)
#     driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    affected_bar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                                    'All Affected Items')))
    affected_bar.click()
    sleep(1)
    driver.switch_to.window(driver.window_handles[-1])
    affected_win = driver.current_window_handle
    return affected_win

In [ ]:
# def message shown palallel
def message(mes):
#     TIME_TO_WAIT = 5000 # in milliseconds 
    root = tk.Tk() 
    root.withdraw()
#     print('print message')
    try:
        print('try to shown message',mes)
        root.after(10000, root.destroy) 
        Message(title="Message", message=mes, master=root).show()
#         print(asdf)
    except: # TclError:
        print('error',mes)
        pass

In [ ]:
def get_url():
    error = False
    url = "https://3dx.kla-tencor.com/3dspace/common/KLA_emxNavigator.jsp?"
    global driver, actions, main_page
    
    try:
        driver = webdriver.Edge()
        actions = ActionChains(driver)
    except:
        tk.messagebox.showinfo(title='Webdriver Error',\
            message='Can not get Web Driver, need to update Driver')
        sys.exit()
    
#     driver = webdriver.Edge()
#     actions = ActionChains(driver)
    main_page = ''
#     try:
    print('try get url')
    driver.get(url)
    print('got the url')
    main_page = driver.current_window_handle
    search_field = WebDriverWait(driver, 150).until(EC.presence_of_element_located((By.XPATH,\
                                                    '//*[@id="quickSearch"]')))
    print('found Search field')
    sleep(1)
    driver.switch_to.window(main_page)
#         driver.maximize_window()

#     except:
#         error = tk.messagebox.askyesno(title='Get url Error',
#         message='Can not go to Enovia, do you want to try again?')
# #             if error == False:
#                 print('dont want to run again')
#                 sys.exit()
#             error = True

    print('return ask to get Url again:',error)
    return error,main_page,driver,actions
#     print('check return at get url')

#         print('got main page',main_page)


print('while loop get url')
error = True
while error:
    error,main_page,driver,actions = get_url()
#     print(error)

In [ ]:
# if eco_num is not None: #go to affected items
# url = "https://3dx.kla-tencor.com/3dspace/common/KLA_emxNavigator.jsp?"
# driver = webdriver.Edge()
# actions = ActionChains(driver)
# try:
#     driver.get(url)
#     main_page = driver.current_window_handle
# except:
#     tk.messagebox.showinfo(title='Error 102', \
#         message='Can not go to Enovia, check your connection')
    

In [ ]:
# from main window, find eco and open
eco_win = ''
if len(main_page) > 5:
    eco_win = get_eco_win(eco_num)

In [ ]:
# from ECO win, click to affected items bar and assign as affected_win
if len(eco_win) > 5:
    affected_win = get_affected_win(eco_win)
    sleep(1)
    driver.switch_to.window(affected_win)
#     driver.maximize_window()

# If request file, then download WUB and write script, if script file selected, then goto ECO

In [ ]:
def collect_add_list(lines):
    
    add_dash = []
    add_rev = []
    add_release = []
    add_inactive = []
    add_check_plant = []
    for line in lines:
        extracts = extract_def(line)
        if len(extracts) < 1:
            continue


        for extract in extracts:
#             print('extract',extract)
            if extract[0] != '':
                if extract[1] == 'dash':
                    if not any(extract[0] in add for add in add_dash):
    #                     print('dash',extract,extract[0])
                        add_dash.append(extract)
                elif extract[1] == 'rev':
                    if not any(extract[0] in add for add in add_rev):
                        add_rev.append(extract)
    #                     print('rev',extract,extract[0])
                elif extract[1] == 'release':
                    if not any(extract[0] in add for add in add_release):
                        add_release.append(extract)
    #                     print('release',extract,extract[0])
                elif extract[1] == 'inactive':
                    if not any(extract[0] in add for add in add_inactive):
                        add_inactive.append(extract)
                elif extract[1] == 'add':
                    if not any(extract[0] in add for add in add_check_plant):
                        add_check_plant.append(extract)
    #                     print('inactive',extract,extract[0])
    print('\nAdd for Dash roll\n',*add_dash, sep = '\n')
    print('\nAdd for Rev roll\n',*add_rev, sep = '\n')
    print('\nAdd for Release\n',*add_release, sep = '\n')
    print('\nAdd for Inactive\n',*add_inactive, sep = '\n')
    print('\nAdd for check plant\n',*add_check_plant, sep = '\n')
    return add_dash,add_rev,add_release,add_inactive,add_check_plant
with open(script_file) as f:
    lines = f.readlines()
add_dash,add_rev,add_release,add_inactive,add_check_plant = collect_add_list(lines)

In [ ]:
# collect BOMs, BOMs structure [[parent - Assy], [child1 - Part1], [child2 - Part2], ... ]
def collect_boms(lines):
    i = 0
    boms = []
    while i < len(lines):
    #     print(i)
        i += 1
        extracts = extract_def(lines[i-1])
        if len(extracts) < 1:
            continue
    #     print(extracts)
        if extracts[0][3] == 'header':
            child_ext = []
            par_ext = []
            if extracts[0][4] == 'child':
                j = i - 1
                while j < len(lines) - 1:
                    j += 1
                    if lines[j][0].isdigit():
                        break
                    if len(extract_def(lines[j])) < 1:
                        continue
#                     print('extract in while')
                    par_ext = extract_def(lines[j])
                    child_ext = extracts
    #                 print('par_ext:',par_ext,'child:',child_ext)
                    boms.append(par_ext + child_ext)
                i = j
            elif extracts[0][4] == 'par':
    #             print('par',extracts)
                for extract in extracts:
                    par_ext = [extract]
                    j = i - 1
                    while j < len(lines) - 1:
                        j += 1
                        if lines[j][0].isdigit():
                            break
                        if len(extract_def(lines[j])) < 1:
                            continue
                        for extract in extract_def(lines[j]):
                            child_ext.append(extract)
    #                 print('par_ext:',par_ext,'child:',child_ext)
    #                 print('combine',par_ext + child_ext)
                    boms.append(par_ext + child_ext)
                    i = j
            elif extracts[0][1] == 'release' and extracts[0][0] == '':
    #             print(extracts[0][0],extracts[0][1])
                j = i - 1
                while j < len(lines) - 1:
                    j += 1
                    if lines[j][0].isdigit():
                        break
                    if len(extract_def(lines[j])) < 1:
                        continue
                    if not any(extract_def(lines[j])[0][0] in add for add in add_release):
                        add_release.append(extract_def(lines[j])[0])
    #                     print('release',extract,extract[0])
                i = j
            elif extracts[0][1] == 'inactive' and extracts[0][0] == '':
                j = i - 1
                while j < len(lines) - 1:
                    j += 1
                    if lines[j][0].isdigit():
                        break
                    if len(extract_def(lines[j])) < 1:
                        continue
                    if not any(extract_def(lines[j])[0][0] in add for add in add_inactive):
                        add_inactive.append(extract_def(lines[j])[0])
                i = j
    
    return boms
boms = collect_boms(lines)
msg_boms = []
for number, bom in enumerate(boms):
    msg_boms.append('_____________________________________________________________')
    msg_boms.append('Bom #' + str(number + 1) + ' Parent is:')
    print('Bom #',number + 1,'Parent is:')
    msg_boms.append(bom[0])
    print(bom[0])
    
    msg_boms.append('Below is childen:')
    print('Below is childen:')
    for i in range(1,len(bom)):
        msg_boms.append(bom[i])
        print(bom[i])
    
    print('')
        
#     if number > 0:
#     print(*bom, sep = '\n')
#     print('')

In [ ]:
# msg_list = [str(x) for x in msg_boms]
# msg_list = '\n'.join(msg_list)
# ask = tk.messagebox.showinfo('Boms list',msg_list)

In [ ]:
# key for add for dash or rev rolled
search_key = []
for part in add_dash:
    print(part[0],'part[0] dash',part[0][:-len(part[0].split('-')[-1]) -1] )
    if not any(part[0][:-len(part[0].split('-')[-1]) -1] in key for key in search_key):
        search_key.append(part[0][:-len(part[0].split('-')[-1]) -1] +'*,')
#         print(search_key)
for part in add_rev:
    print(part[0],'part[0] rev',part[0][:-len(part[0].split('-')[-1]) -1] )
    if not any(part[0][:-len(part[0].split('-')[-1]) -1] in key for key in search_key):
        search_key.append(part[0][:-len(part[0].split('-')[-1]) - 1] +'*,')
print('key search for affected items:')
print(*search_key,sep='\n')
    

In [ ]:
# refresh page
def refr_pg():
    print('refresh page ...')
    driver.switch_to.window(affected_win)
#     driver.maximize_window()
#     wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    try:
        driver.switch_to.default_content()
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/div[3]/div/div/div[2]/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/span/table/tbody/tr/td[4]/div/a/button')))
        refresh.click()
        print('refreshed')
    
    except:
        try:
            driver.switch_to.default_content()
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
            driver.switch_to.frame('detailsDisplay')
            driver.switch_to.frame('tableContentFrame')
            refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
                 '//*[@id="toolbar-container"]/div/div/div[4]/i')))
            refresh.click()
            print('refreshed')
        except:
            pass
#         refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
#         '/html/body/div[3]/div/div/div[2]/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/span/table/tbody/tr/td[4]/div/a/button')))
#         refresh.click()
#         print('refreshed')
#     print('test error')
#     print(error)
    
    sleep(1)

In [ ]:
add_release

In [ ]:
search_key

In [ ]:
def add_items():
# check if part added to affected items
    driver.switch_to.window(affected_win)
#     driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    # driver.switch_to.frame('content')
    affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                            '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
    affected_items_bar.click()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    print('check if part added to affected items')
    for i in range(len(search_key)-1,-1,-1):
        for item in affected_items:
            if search_key[i][:7] in item.text:
                print(search_key[i],'has been added to affected items')
                search_key.pop(i)
                break
    
    for i in range(len(add_release)-1,-1,-1):
        for item in affected_items:
            if add_release[i][0][:7] in item.text:
                print(add_release[i],'has been added to release')
                add_release.pop(i)
                break
    
    if len(search_key) > 0:
        print('search_key remain',search_key)

        # need to deal with more than 150 items
        driver.switch_to.window(affected_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        actions_btn.click()
        rev_dash_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Add For Revise/Dashroll')))
        print('got Add For Revise/Dashroll')
        sleep(random.randint(0, 2))
        rev_dash_btn.click()

        # input search key
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        names_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/form/table/tbody/tr[3]/td[2]/table/tbody/tr/td[1]/textarea')))
        names_input.clear()
        names_input.send_keys(search_key)
        print('key has input')
        sleep(random.randint(0, 2))
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')
        find_affected_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]')))
        print('find_affected_btn found')
        sleep(random.randint(0, 2))
        find_affected_btn.click()

        # pagination
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')

        pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
        pagination.click()

        print('select correct items')
        driver.switch_to.window(driver.window_handles[3])
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        driver.switch_to.frame('tableBodyRight')
        rerult_list = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
        print('found total',len(rerult_list),'items')

        s = 0
    #     add_release = []
    #     add_rev_dash = []

        for number, part in enumerate(add_dash):
            break_part = False   

            # find add for release dash roll
            for i in range(len(rerult_list)):
                if part[2] in rerult_list[i].text:# and 'preliminary' in rerult_list[i].text.lower():
                    add_dash.pop(number)
                    if not any(part[2] in item for item in add_release):
                        add_release.append(part)
                        print(part, 'added for release')
                    break_part = True
                    break
            if break_part:
                continue
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text:
    #                 add_rev_dash.append(part)
                    s += 1
                    part_found = driver.find_element(By.XPATH,\
                            '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                    actions.move_to_element(part_found).perform()
                    part_found.click()
                    print(part,'selected')                
        for number, part in enumerate(add_rev):
            break_part = False 
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text\
                and part[2] in (rerult_list[i].text[rerult_list[i].text.index(part[0])+11\
                                    :rerult_list[i].text.index(part[0])+15]).strip():
    #                 print(rerult_list[i].text[rerult_list[i].text.index(part[0])+11\
    #                                 :rerult_list[i].text.index(part[0])+15])
                    add_rev.pop(number)

                    if not any(part[0] in item for item in add_release):
                        print(part, 'added for release')
                        add_release.append(part)
                    break_part = True
                    break
            if break_part:
                continue
    #                 # found curent rev
            new_rev = part[2]
            letters = string.ascii_uppercase
            if new_rev[-1] == 'A' and new_rev[0] != 'A':
                if new_rev[0] == 'J' or new_rev[0] == 'P' or new_rev[0] == 'R'\
                          or new_rev[0] == 'T' or new_rev[0] == 'Y':
                    index = letters.index(new_rev[0])
                    cur_rev = letters[index - 2] + 'Y'   
                else:
                    index = letters.index(new_rev[0])
                    cur_rev = letters[index - 1] + 'Y'   
            elif new_rev[0] == 'J' or new_rev[0] == 'P' or new_rev[0] == 'R'\
                          or new_rev[0] == 'T' or new_rev[0] == 'Y':
                index = letters.index(new_rev[-1])
                cur_rev = new_rev[0] + letters[index - 2] 
            else:
                index = letters.index(new_rev[-1])
                cur_rev = new_rev[0] + letters[index - 1] 

            # found correct part to up rev
            for i in range(len(rerult_list)):
                if part[0] in rerult_list[i].text\
                and cur_rev in rerult_list[i].text[rerult_list[i].text.index(part[0])+11:rerult_list[i].text.index(part[0])+15]:
                    s += 1
    #                 add_rev_dash.append(part)
                    part_found = driver.find_element(By.XPATH,\
                            '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                    actions.move_to_element(part_found).perform()
                    part_found.click()
                    print(part, 'selected')
        if s > 0:
            ask = tk.messagebox.askokcancel('Confirm',\
                'Please check: ' + str(s) + ' part(s) selected to add to Affected items. Click Select, then click Ok to continue or Cancel to quit')
            if ask == False:

                sys.exit()
        #     print('selected total:',s,'items')
            print('below part need to add for release:')
            print(*add_release, sep = '\n')

    # go to add for release items, if len(add_release) > 0:
    if len(add_release) > 0:
        print(str(len(add_release)) + ' parts need to add for release')
#     else:
        driver.switch_to.window(affected_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        rev_dash_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Add For Release')))
        print('got Add For Release')
        sleep(random.randint(0, 2))
        rev_dash_btn.click()

        # input search key
    #     driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        names_input1 = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/div[1]/div/div[3]/form/table/tbody/tr[6]/td/table/tbody/tr/td[1]/textarea')))
        print('names_input found')
        names_input1.clear()
        for part in add_release:
            names_input1.send_keys(part[0][:7] + '*\n')
        # names_input.send_keys(search_key)
        # print('names_input has input')
        submit_btn1 = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                        '//*[@id="SUBMIT_BUTTONS"]/a[2]')))
        submit_btn1.click()

        #pagination
        driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableBottomFrame')

        pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
        pagination.click()

        # select part for release
    #     driver.switch_to.window(driver.window_handles[2])
        driver.switch_to.default_content()
        driver.switch_to.frame('content')
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tvcInlineFrame_-1')
        driver.switch_to.frame('searchContentFrame')
        driver.switch_to.frame('tableContentFrame')
        driver.switch_to.frame('tableBodyRight')
        items_found = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')

        # if len(add_release) > 0:
        pn_release_selected = 0
        for part in add_release:
            if part[1] == 'dash':
                part_find = part[2]
            else:
                part_find = part[0]
            print('finding part:',part)
            for i in range(len(items_found)):
                if part_find in items_found[i].text:

                    try:
                        select_part = driver.find_element(By.XPATH,\
                                '/html/body/form/div[2]/div/table/tbody/tr[' + str(i+1) + ']/td/table/tbody/tr/td[2]/input')
                        actions.move_to_element(select_part).perform()
                        select_part.click()
                        print(i,part_find,'found --- selected',items_found[i].text)
                        pn_release_selected += 1
                    except:
                        print('part found but can not selected')
        if pn_release_selected > 0:
            ask = tk.messagebox.askokcancel('Confirm','Please check, Submit parts add for release, then click Ok to continue or Cancel to quit')
            if ask == False:
                print('exit program')
                sys.exit()

    # switch to affected items window
    driver.switch_to.window(affected_win)
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    # print('switched to frame content')
    # affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
    #                                         '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
    # print('affected_items btn found')
    # affected_items_bar.click()
    # driver.switch_to.window(driver.window_handles[2])
    # driver.switch_to.frame('content')
    driver.switch_to.frame('detailsDisplay')
    driver.switch_to.frame('tableBottomFrame')
    pagination = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                '//*[@id="tvcPageControlOff"]')))
    pagination.click()

    # refresh page
    print('try to refresh page ...')
    refr_pg()
    
#     driver.switch_to.window(affected_win)
# #     driver.maximize_window()
#     wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
#     driver.switch_to.frame('detailsDisplay')
#     driver.switch_to.frame('tableContentFrame')
#     try:
#         refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
#              '//*[@id="toolbar-container"]/div/div/div[4]/i')))
#     except:
#         driver.switch_to.default_content()
#         wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
#         refresh = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,\
#         '/html/body/div[3]/div/div/div[2]/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/span/table/tbody/tr/td[4]/div/a/button')))


#     refresh.click()
#     print('refreshed')
#     sleep(1)
    
#     print(error)
    # work with affected window
    driver.switch_to.window(affected_win)
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    # sleep(2)
    
    # add for dash rolled first
    s = 0
    for part in add_dash:
    #     print('part',part)
        for i in range(len(affected_items)):
            cur_p_dash = driver.find_element(By.XPATH,\
            '/html/body/form/div[3]/div[2]/table/tbody/tr[' + str(i+1) + ']/td[3]/a')
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[13]')
            new_p_dash = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')

            if uom.text != '' and cur_p_dash.text == part[0] and new_p_dash.text == '':
                print('cur_p_dash',cur_p_dash.text)
                print('uom',uom.text)
                print('new_p_dash',new_p_dash.text)
    #         /html/body/form/div[3]/div[2]/table/tbody/tr[31]/td[3]/a
    #         if len(re.findall(part[2][:7], affected_items[i].text)) > 1:
    #             print('part has up roll')
    #         elif len(re.findall(part[0], affected_items[i].text)) == 1:
    # #             print(re.findall(part[0], affected_items[i].text))
    # #             print(len([re.findall(part[0], affected_items[i].text)]))
                part_dash = driver.find_element(By.XPATH,\
                        '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                actions.move_to_element(part_dash).perform()
                part_dash.click()
                s += 1


    # up dash roll selected
    if s > 0:
        driver.switch_to.window(affected_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        dash_selected = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Dashroll Selected')))
        print('Dashroll Selected')
        sleep(random.randint(0, 2))
        dash_selected.click()
        ask = tk.messagebox.askokcancel('Confirm','Please select Dashroll Selected, confirm, then click Ok to continue or Cancel to quit')
        if ask == False:
            print('exit program')
            sys.exit()


    # find part for rev roll
    driver.switch_to.window(affected_win)
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
    affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
    sleep(2)
    # add for rev rolled rolled first
    r = 0
    for part in add_rev:
    #     print('part',part)
    #     print('range(len(affected_items))',range(len(affected_items)))
        for i in range(len(affected_items)):
    #         part_rev= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,\
    #                             '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[18]/a')))
            new_pn = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')
            cur_pn = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[3]')
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[13]')
    #         /html/body/form/div[3]/div[2]/table/tbody/tr[5]/td[18]/a
    #         part_rev = driver.find_element(By.XPATH,\
    #                     '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[18]')
    #         print('part_rev',part_rev_text.text)
    #         print('affected_items[i].text',affected_items[i].text)
    #         print(len(re.findall(part[0][:7], affected_items[i].text)))
    #         if len(re.findall(part[0][:7], affected_items[i].text)) > 1:
            if cur_pn.text == part[0] and new_pn.text == '' and uom.text != '':

    #         elif len(re.findall(part[0][:7], affected_items[i].text)) == 1:
                print(re.findall(part[0], affected_items[i].text),'has selected')
    #             print(len([re.findall(part[0], affected_items[i].text)]))
                part_rev = driver.find_element(By.XPATH,\
                        '//*[@id="tbl"]/tbody/tr[' + str(i+1) + ']/td[2]/input[1]')
                actions.move_to_element(part_rev).perform()

                part_rev.click()
                r += 1
            elif new_pn.text == part[0] and uom.text != '':
                print(part[0],' has up rev roll')

    print('has',r,'need to up rev')

    # up rev roll selected, dont need to confirm
    if r > 0:
        driver.switch_to.window(affected_win)
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        driver.switch_to.frame('detailsDisplay')
        driver.switch_to.frame('tableContentFrame')
        actions_btn = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                                '//*[@id="toolbar-container"]/div/div/div[1]/div[1]/span')))
        print('actions button found')
        actions_btn.click()
        print('actions button click')
        rev_selected = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,\
                            'Revise Selected')))
        print('Revise Selected')
        sleep(random.randint(0, 2))
#     if r > 0:
        ask = tk.messagebox.askokcancel('Confirm','Please select rev rolled selected parts, then click Ok to continue or Cancel to quit')
        if ask == False:
            print('exit program')
            sys.exit()
#     else:
#     #     rev_selected.click()
#         print('parts has up rev rolled')
        
        



In [ ]:
def add_item_loop():
#     global error
    error = False
    try:
        print('Try run add_items() function')
        
        mes = 'adding affected items'
        thread = threading.Thread(target = message,args=(mes, ))
        thread.start()
        add_items()
        print('____________________ Suppress: Executed adding items to ECO Successfully ___________________')
    except:
#         mes = 'Error on Add Affected items and dash/rev rolled, go to next'
#         thread = threading.Thread(target = message,args=(mes, ))
#         thread.start()
        error = tk.messagebox.askyesno(title='Add items Error',
            message='Can not add affected items, do you want to run again?')
    return error
error = True
while error:
        error = add_item_loop()
print('Executed add items loop')

In [ ]:
boms

In [ ]:
# for bom in boms:
#     print(bom)

In [ ]:
# update BOMs
def update_boms():
#     go to Affected bar
    driver.switch_to.window(affected_win)
    sleep(2)
#     driver.maximize_window()
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    # driver.switch_to.frame('content')
    affected_items_bar = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
                                            '//*[@id="tvcTabs0"]/div[1]/ul/li[2]/div/a[1]')))
    affected_items_bar.click()

    # update BOMs
    # for number, part in enumerate(add_dash):
    first_assy = True
    for number, bom in enumerate(boms):
    #     print('number ',number+1,' of ', len(boms))
    # def update_bom(affected_win, bom, thread_name):
        print(f'go to udpate for BOM {number+1}/{len(boms)}: {bom[0]}')
        driver.switch_to.window(affected_win)
#         driver.maximize_window()
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
        affected_items = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')

        if bom[0][1] == 'dash':
            assy_find = bom[0][2]
            rev_find = 'AA'
        elif bom[0][1] == 'rev':
            assy_find = bom[0][0]
            rev_find = bom[0][2]
        else:
            print('can not find assy and rev to find, break at extract',bom[0])
            continue #pass #return
        for i in range(len(affected_items)):
            driver.switch_to.window(affected_win)
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
            wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
            uom = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[28]')
            new_assy = driver.find_element(By.XPATH, '/html/body/form/div[3]/div[2]/table/tbody/tr['\
                                + str(i+1) + ']/td[18]')
            # just find key on name in new rev/doash rolled, do not check the dash rolled
            if uom.text != '' and new_assy.text.split('-')[0] == assy_find.split('-')[0] :
    #             first_assy = True
                open_in = driver.find_element(By.XPATH,\
                            '//*[@id="tbr"]/tbody/tr[' + str(i+1) + ']/td[16]/a/img')
                actions.move_to_element(open_in).perform()
                open_in.click()
                print('found assy ______:',assy_find,'item',i,'of',len(affected_items))
                sleep(2)
                driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
                bom_win = driver.current_window_handle      
#                 driver.maximize_window()
                if first_assy:
                    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                    ebom_btn = WebDriverWait(driver, 2).until\
                         (EC.presence_of_element_located((By.XPATH,\
                            '/html/body/div[3]/div/div/div[3]/div[1]/ul/li[2]/div/a[1]')))
                    actions.move_to_element(ebom_btn).perform()
                    ebom_btn.click()
                    first_assy = False
                    print('first_assy has change to:',first_assy)
                driver.switch_to.window(bom_win)
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                part_in_bom = driver.find_elements(By.XPATH, '//*[@id="tbr"]/tbody/tr')
                child_count = len(part_in_bom)
                print('count parts in BOM:',child_count)
                for j in range(1, len(bom)):

                    child = bom[j]
                    print('updating for child: ',child)
                    if child[1] == 'rev'\
                    or child[1]  == 'dash'\
                    or child[1] == 'replace':
                        if child[1] == 'dash':
                            part_rep = child[2]
                            rev_rep = 'AA'
                        elif child[1] == 'replace':
                            part_rep = child[2]
                            rev_rep = ''
                        else:
                            part_rep = child[0]
                            rev_rep = child[2]
                        # sleep(1)
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
    #                         print('k:',k)
    #                         driver.switch_to.window(bom_win)
    #                         wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    #                         wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
    #                         wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
    #                         wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            rev_child = driver.find_element(By.XPATH,\
                                    '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[3]')
    #                         print('PN and Rev:',pn_child.text,rev_child.text)
                            if len(rev_child.text) > 6 and rev_child.text[-6] == '.' \
                                and pn_child.text == part_rep:
#                             if pn_child.text.split('-')[-1].isnumeric()\
#                                 and part_rep.split('-')[-1].isnumeric()\
#                                 and (int(pn_child.text.split('-')[-1]) >= int(part_rep.split('-')[-1])):
#                                 print(child, 'has been replaced, break to next child')
                                break
#                             elif (child[1] == 'dash' and child[2] == pn_child.text)\
#                                 or (child[1] == 'replace' and child[2] == pn_child.text):
#                                 print('Child Dash rolled or replace has been replaced',pn_child.text)
#                                 break
                            elif (child[1] == 'dash' and child[2].split('-')[0] == pn_child.text.split('-')[0])\
                                and pn_child.text.split('-')[-1].isnumeric()\
                                and child[2].split('-')[-1].isnumeric()\
                                and (int(pn_child.text.split('-')[-1]) >= int(child[2].split('-')[-1])):
                                print('Child Dash rolled has been replaced',pn_child.text)
                                break
    
                            elif (child[1] == 'replace' and child[2].split('-')[0] == pn_child.text.split('-')[0])\
                                and pn_child.text.split('-')[-1].isnumeric()\
                                and child[2].split('-')[-1].isnumeric()\
                                and (int(pn_child.text.split('-')[-1]) >= int(child[2].split('-')[-1])):
                                print('Child Dash rolled or replace has been replaced',pn_child.text)
                                break
                    
                            elif child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child:',child)
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()
                                except:
                                    print('part found but can not selected, break')
                                    break
                                driver.switch_to.window(bom_win)
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                actions_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                                actions.move_to_element(actions_btn).perform()
                                actions_btn.click()
                                try:
                                    rep_with_existing = WebDriverWait(driver, 2).until\
                                        (EC.presence_of_element_located((By.LINK_TEXT,\
                                                'Replace with Existing')))
                                    rep_with_existing.click()
                                except:
                                    print('can not select button rep_with_existing for',child)
                                    break
                                driver.switch_to.window(driver.window_handles\
                                            [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                name_input = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr/td[1]/textarea')))
                                name_input.send_keys(part_rep[:7] + '*')
                                driver.switch_to.window(driver.window_handles\
                                                        [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBottomFrame'))
                                find_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '/html/body/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]/a')))
                                actions.move_to_element(find_btn).perform()
                                find_btn.click()
                                # go to result of finding
                                driver.switch_to.window(driver.window_handles\
                                                        [len(driver.window_handles)-1])
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('searchContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                                parts_found = driver.find_elements(By.XPATH,\
                                                                    '//*[@id="tbr"]/tbody/tr')
                                # select last part in list, need to check
                                select_part = driver.find_element(By.XPATH,\
                                        '//*[@id="tbl"]/tbody/tr['\
                                                + str(len(parts_found)) + ']/td[1]/input')
                                actions.move_to_element(select_part).perform()
                                select_part.click()
                                print('part selected:',child[0])

                                ask = tk.messagebox.askokcancel('Confirm',\
                                'Please Submit Replace Part, then click Ok to continue or Cancel to quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    driver.switch_to.window(driver.window_handles[\
                                                    len(driver.window_handles)-1])
                                    driver.close() 
                                    sys.exit()
                                driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                if driver.current_window_handle != affected_win\
                                    and driver.current_window_handle != bom_win:
                                    print('window is not BOM-window, closing')
                                    driver.close()    

                                # break after replace child to move next child
                                break
                    elif child[1] == 'add': # change to elif after testing                    
                        driver.switch_to.window(bom_win)    
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        part_added = False
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('Part has been added ',pn_child.text)
                                part_added = True
                                break
                        if part_added:
                            continue
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                        select_all.click()
                        select_all.click()
                        driver.switch_to.window(bom_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))



                        actions_btn = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                        actions.move_to_element(actions_btn).perform()
                        actions_btn.click()
                        try:
                            add_existing = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.LINK_TEXT,\
                                        "Add Existing [Drag'n Drop]")))
                            add_existing.click()
                        except:
                            print('can not select btn add_existing for',child_extract[0][0])
                            continue
                        driver.switch_to.default_content()
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        driver.switch_to.frame('tvcInlineFrame_-1')
                        driver.switch_to.frame('searchContentFrame')
                        input_area = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                        '//*[@id="tvcSearchForm"]/table/tbody/tr[4]/td/table/tbody/tr/td[1]/textarea')))
                        input_area.send_keys(child[0][:7] + '*')
                        submit_btn = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.LINK_TEXT,\
                                        "Submit")))
                        submit_btn.click()
                        driver.switch_to.default_content()
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        driver.switch_to.frame('tvcInlineFrame_-1')
                        driver.switch_to.frame('searchContentFrame')
                        driver.switch_to.frame('tableContentFrame')
                        driver.switch_to.frame('tableBodyRight')
                        tick_box = WebDriverWait(driver, 2).until\
                                (EC.presence_of_element_located((By.XPATH,\
                            '/html/body/form/div[2]/div[1]/table/tbody/tr/td[1]/table/tbody/tr/td[2]/input[1]')))
                        tick_box.click()
                        ask = tk.messagebox.askokcancel('Confirm','Please check Added Part, then drag to BOM')
                        if ask == False:
                            print('exit program')
                            driver.switch_to.window(driver.window_handles[\
                                                len(driver.window_handles)-1])
                            driver.close()  
                            sys.exit()
                        print('found part to add: ',child)
                        continue
                    elif child[1] == 'remove':
                        driver.switch_to.window(bom_win)
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child to remove:',child[0])
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                print('select_all')
                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()
                                except:
                                    print('part found but can not selected')
                                    break
                                driver.switch_to.window(bom_win)
                                driver.switch_to.frame('content')
                                driver.switch_to.frame('detailsDisplay')
                                driver.switch_to.frame('tableContentFrame')
                                actions_btn = WebDriverWait(driver, 2).until\
                                    (EC.presence_of_element_located((By.XPATH,\
                                        '//*[@id="toolbar-container"]/div/div/div[4]/div[1]/span')))
                                actions.move_to_element(actions_btn).perform()
                                actions_btn.click()

                                # select replace with existing part
                                try:
                                    rep_with_existing = WebDriverWait(driver, 2).until\
                                        (EC.presence_of_element_located((By.LINK_TEXT,\
                                                'Remove Selected')))
                                    rep_with_existing.click()
                                except:
                                    print('can not select for Remove Selected',child)
                                    break
                                ask = tk.messagebox.askokcancel('Confirm',\
                                    'Please check Part to remove, then confirm and click Ok to continue or cancel to Quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    sys.exit()

                                break

                    elif child[1] == 'change':
                        driver.switch_to.window(bom_win)
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                        wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
                        for k in range(child_count):
                            pn_child = driver.find_element(By.XPATH, \
                                '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                        + str(k+1) + ']/td[4]/table/tbody/tr[2]')
                            if child[0][:7] in pn_child.text: #part_in_bom[k].text:
                                print('found child to change qty:',child[0])
                                select_all =  driver.find_element(By.XPATH,\
                                '/html/body/form/div/div/table/thead/tr[3]/th[2]/input')
                                select_all.click()
                                select_all.click()
                                print('select_all')

                                try:
                                    select_part = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div/table/tbody/tr['\
                                                    + str(k+1) + ']/td[2]/input[1]')
                                    actions.move_to_element(select_part).perform()
                                    select_part.click()

    #                                 /html/body/form/div[2]/div[2]/table/tbody/tr[31]/td[8]
                                    child_qty = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                                    + str(k+1) + ']/td[8]')

                                    print('current Qty', child_qty.text)
    #                                     print(type(child_qty.text))
                                    if child_qty.text[:len(child[2])] == child[2]:
                                        print('Qty has changed before, break to next')
                                        break
                                    actions.move_to_element(child_qty).perform()
                                    child_qty.click()
    #                                 '/html/body/form/div[2]/div[2]/table/tbody/tr['\
    #                                                 + str(k+1) + ']/td[8]/input'
                                except:
                                    print('part found but can not selected')
                                    break



                                ask = tk.messagebox.askokcancel('Confirm',\
                                    'Please input Qty '+child[2]+' for '\
                                        +pn_child.text+', then click Ok to continue or cancel to Quit')
                                if ask == False:
                                    print('exit program')
                                    driver.switch_to.window(driver.window_handles[\
                                                        len(driver.window_handles)-1])
                                    driver.close()  
                                    sys.exit()
                                space_click = driver.find_element(By.XPATH,\
                                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                                    + str(k+1) + ']/td[9]')
                                actions.move_to_element(space_click).perform()
                                space_click.click()
                                break
                print('closing ... at:',j+1,'of', len(bom))
                sleep(1)
                driver.switch_to.window(bom_win)
                if driver.current_window_handle != affected_win:
                    print('window is not affected_win, closing')
                    driver.close()    
                # break to next assy
#                 break

    print('done')

In [ ]:
# try refresh page
try:
    print('Try run refr_pg() function')
    refr_pg()
    
    print('____________________ Suppress: Executed page refresh Successfully __________________________')
except:
    mes = 'Error on refresh Page, go to next'
    thread = threading.Thread(target = message,args=(mes, ))
    thread.start()
#     thread.join()
#     print('Error on refresh Page, go to next')
    

In [ ]:
def update_bom_loop():
    error = False
    try:
        
        mes = 'Executing BOMs updating'
        thread = threading.Thread(target = message,args=(mes, ))
        thread.start()
        print('Try run update_boms() function')
        update_boms()
        print('____________________ Suppress: Executed BOMs Updating Successfully _________________________')
    except:
        mes = 'Error on update BOM'
        thread = threading.Thread(target = message,args=(mes, ))
        thread.start()
        error = tk.messagebox.askyesno(title='Update BOM Error',
            message='Can not update BOMs, do you want to run again?')
    return error
error = True
while error:
        error = update_bom_loop()
print('Executed update BOMs loop')

# sys.exit()

In [ ]:
plant_opt = '3008'

# def update_SAP(plant_opt):
# check if part added to affected items
driver.switch_to.window(affected_win)
# affected_win = driver.current_window_handle
#     driver.maximize_window()
wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
# driver.switch_to.frame('content')
SAP_ID = WebDriverWait(driver, 20).until(EC.presence_of_element_located((\
            By.LINK_TEXT,'SAP Initial Data')))
actions.move_to_element(SAP_ID).perform()
SAP_ID.click()

# update SAP ID
driver.switch_to.window(affected_win)
wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
SAP_items = driver.find_elements(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr')
for i in range(len(SAP_items)):
    seq = driver.find_element(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                    + str(i+1) +']/td[3]')

    # go to update SAP ID for part with seq == 1
    if seq.text == '1':
        new_pn = driver.find_element(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                    + str(i+1) +']/td[2]')
        print('Found Part seq.',seq.text,new_pn.text)

        ### collect info. through old part
        found_old_pn = False
        for j in range(len(SAP_items)):
#             print(j,len(SAP_items))
            old_pn = driver.find_element(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                    + str(j+1) +']/td[2]')


            if old_pn.text[:7] == new_pn.text[:7] and j != i:
                #found old part
                found_old_pn = True
                print(f'found old_pn {old_pn.text} at {j}')
                old_pn_data = driver.find_element(By.XPATH,\
                    '/html/body/form/div[2]/div[1]/table/tbody/tr['+ str(j+1) +']/td[2]/a/img')
#                 /html/body/form/div[2]/div[1]/table/tbody/tr[13]/td[2]/a/img
                # move to old part and click()
                actions.move_to_element(old_pn_data).perform()
                old_pn_data.click()
                driver.switch_to.window(driver.window_handles\
                    [len(driver.window_handles)-1])
                old_data_win = driver.current_window_handle
#                 driver.maximize_window()
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('content'))
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('pagecontent'))
                SAP_plants = driver.find_elements(By.XPATH,\
                        '/html/body/div[6]/table/tbody/tr')
                print('SAP data table row:',len(SAP_plants))
                old_data = []
                for k in range(len(SAP_plants)):
                    # check the row is visible or not, the craw data
                    if SAP_plants[k].get_attribute('class') == 'visibleRow':
                        plant_val = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[1]').text
                        proc_type = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[4]').text
                        in_house_pro = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[8]').text
                        pln_del_time = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[9]').text
                        pur_grp = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[10]').text
                        pur_name = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[11]').text
                        mrp_cont = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[12]').text
                        mrp_name = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[13]').text
                        std_price = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[15]').text
                        pln_pri_1 = driver.find_element(By.XPATH, \
                            '/html/body/div[6]/table/tbody/tr['\
                            + str(k+1) +']/td[16]').text
                        old_data.append([plant_val,proc_type,in_house_pro,pln_del_time,\
                            pur_grp,pur_name,mrp_cont,mrp_name,std_price,pln_pri_1])

#                 sleep(5)
                driver.close()

                driver.switch_to.window(affected_win)
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
                wait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it('tableBodyRight'))
# #                 SAP_items = driver.find_elements(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr')
                break
        if found_old_pn == False:
            continue
        print(f'{old_data} found at {j+1}')

        #### update for old part
        old_pn_plant_val = driver.find_element(By.XPATH, \
                        '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) +']/td[11]')
        actions.move_to_element(old_pn_plant_val).perform()
        old_pn_plant_val.click()
        old_pn_plant_val.click()
        print('list',old_pn_plant_val.text)
    # click select 30-inactive
        try:
            driver.find_element(By.XPATH, \
                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) +']/td[11]/select/option[7]').click()
        except:
            old_pn_plant_val.click()
            driver.find_element(By.XPATH, \
                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j+1) +']/td[11]/select/option[7]').click()
        print('len(old_data)',len(old_data))


        # loop in rows plant

        if len(old_data) > 0:
            for row in old_data:
                if row[0] == plant_opt:
                    print(f'{row} has plant = {plant_opt}')
                    
                    ## update for new part _________
                    new_plant_val = driver.find_element(By.XPATH, \
                                    '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                    + str(i+1) +']/td[11]')
                    actions.move_to_element(new_plant_val).perform()
                    new_plant_val.click()
                    new_plant_val.click()

                    print('list',new_plant_val.text)
                    try:
                        driver.find_element(By.XPATH, \
                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                            + str(i+1) +']/td[11]/select/option[3]').click()
                    except:
                        print('except select list plant value')
                        new_plant_val.click()
                        driver.find_element(By.XPATH, \
                            '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                            + str(i+1) +']/td[11]/select/option[3]').click()
                # click to space pos
                    space_click = driver.find_element(By.XPATH, \
                                    '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                    + str(i+1) +']/td[3]')
                    actions.move_to_element(space_click).perform()
                    space_click.click()
                # update make/by

                    ## update lead time

                    # update MRP controller
#                     MRP_controller_area = driver.find_element(By.XPATH, \
#                                     '/html/body/form/div[2]/div[2]/table/tbody/tr['\
#                                     + str(i+1) +']/td[16]')

#                     actions.move_to_element(MRP_controller_area).perform()
#                     MRP_controller_area.click()
#                     MRP_controller_area.click()
#                     try:
#                         MRP_controller_input = driver.find_element(By.XPATH, \
#                                         '/html/body/form/div[2]/div[2]/table/tbody/tr['\
#                                         + str(i+1) +']/td[16]/input')
#                         ### /html/body/form/div[2]/div[2]/table/tbody/tr[5]/td[16]/input
#                         print('MRP_controller_input',MRP_controller_input.text)
#                         MRP_controller_input.clear()
#                         MRP_controller_input.send_keys(row[5])
#                         sleep(5)
#                     except:
#                         MRP_controller_area.click()
#                         MRP_controller_input = driver.find_element(By.XPATH, \
#                                         '/html/body/form/div[2]/div[2]/table/tbody/tr['\
#                                         + str(i+1) +']/td[16]/input')
#                         ### /html/body/form/div[2]/div[2]/table/tbody/tr[5]/td[16]/input
#                         print('MRP_controller_input',MRP_controller_input.text)
#                         MRP_controller_input.clear()
#                         MRP_controller_input.send_keys(row[5])
#                         sleep(5)
#                     auto_complete = driver.find_element(By.XPATH, \
#                             "//input[starts-with(@class, 'ac_cell')]")
#                     sleep(5)
#                     auto_complete[0].click()
#                     sleep(5)
                        
#                     ask = tk.messagebox.askokcancel('Confirm',\
#                     'Please Submit Replace Part, then click Ok to continue or Cancel to quit')
#                     if ask == False:
#                         continue
                        
#                         except:
#                             print('cannot clear')
                            
                    break
                    


        break

# update_SAP(plant_opt)
# print('done')

In [ ]:
def update_SAP_ID_loop():
    error = False
    try:
        print('Try run update_SAP_ID_loop() function')
        update_SAP()
        mes = 'Executing SAP ID updating'
        thread = threading.Thread(target = message,args=(mes, ))
        thread.start()
        print('____________________ Suppress: Executed SAP_ID Updating Successfully _________________________')
    except:
        mes = 'Error on update SAP ID'
        thread = threading.Thread(target = message,args=(mes, ))
        thread.start()
        error = tk.messagebox.askyesno(title='Update SAP ID Error',
            message='Can not update SAP ID, do you want to run again?')
    return error
error = True
while error:
        error = update_SAP_ID_loop()
print('Executed update SAP ID loop')

# obsolete